In [ ]:
from main import main

main()

###  cookes categoria 

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pandas as pd
import re
engine = create_engine("sqlite:///produtos.db")
Session = sessionmaker(bind=engine)
session = Session()

query = """
SELECT link_imagem 
FROM imagens;
"""
df = pd.read_sql_query(query, engine)


# Função para extrair sufixo e extensão
def extrair_sufixo_extensao(url):
    match = re.search(r'/(\d+)(_.*?)(\.\w+)(\?|$)', url)
    if match:
        return match.group(2), match.group(3)  # Retorna o sufixo e a extensão
    return None, None

# Aplicar a função e expandir para colunas separadas
df[['Sufixo', 'Extensao']] = df['link_imagem'].apply(lambda x: pd.Series(extrair_sufixo_extensao(x)))
#display(df[['Sufixo', 'Extensao']].value_counts())
#display(df[['Sufixo']].value_counts())
#display(df[[ 'Extensao']].value_counts())